# Scrape Data from Shopee with beautifulsoup and selenium
### Data เพื่อการทำ Marketing ของผู้ขาย ซึ่งประกอบไปด้วย
- name, review, rating, date

!pip install selenium
!pip install bs4
!pip install pandas

# Code ที่ต้อง Import

In [3]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import os  
from datetime import datetime 
import bs4
import time
import re
import random

In [48]:
#กำหนดตัวแปร
#link ที่ต้องการดึงข้อมูล
link = 'https://www.lazada.co.th/products/biore-uv-aqua-rich-watery-gel-spf50pa-90-i598288.html?spm=a2o4m.searchlist.list.53.23e73203ARcL7e'
#ชื่อ file output ใส่ชื่อผลิตภัณฑ์
output = 'Biore UV Aqua Rich Watery Gel'
# Set up Chrome binary_location
location = 'C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe'
#the service path to ChromeDriver
service_path = r'D:\Git\Text-Sentiment-Analysis\chromedriver.exe'

In [49]:
# Set up Chrome options
options = Options()
options.binary_location = location

# Specify the path to ChromeDriver (NOT geckodriver, which is for Firefox)
service = Service(service_path)

# Initialize WebDriver with the service and options
driver = webdriver.Chrome(service=service, options=options)

# Open lazada website
driver.get(link)

#wait foe 35s.

In [50]:
#Zoom out เพื่อให้ load ทั่วหน้าจอ
driver.execute_script("document.body.style.zoom='10%'")

In [16]:
records=[]

In [53]:
# การ Scrape ข้อมูลจากหลายหน้าของ Web Shopee
for i in range(3):
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)
    el=soup.select("#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div")
    # Define star rating image sources
    src_plus_one = "//img.lazcdn.com/g/tps/tfs/TB19ZvEgfDH8KJjy1XcXXcpdXXa-64-64.png"
    src_plus_zero = "//img.lazcdn.com/g/tps/tfs/TB18ZvEgfDH8KJjy1XcXXcpdXXa-64-64.png"

    # Extract shop 
    shop_element = soup.select_one("#module_seller_info > div > div.seller-name-retail > div.seller-name__wrapper > div.seller-name__detail > a.pdp-link.pdp-link_size_l.pdp-link_theme_black.seller-name__detail-name")
    shop = shop_element.text.strip() if shop_element else ""

    for e in el:

        # Extract review text
        review = e.select_one("#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div > div.item-content > div.content")
        review_text = review.text.strip() if review else ""

        # Extract and calculate rating
        stars = e.select_one('#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div > div.top > div')
        rating = 0
        if stars:
            for star in stars.find_all('img'):
                if star['src'] == src_plus_one:
                    rating += 1
                elif star['src'] == src_plus_zero:
                    rating += 0

        # Extract review date
        date_element = e.select_one('#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div > div.top > span')
        date = date_element.text.strip() if date_element else ""

        # Extract reviewer name
        name_element = e.select_one('#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div > div.middle > span:nth-child(1)')
        name = name_element.text.strip() if name_element else ""

        # Append the extracted data to records
        records.append([shop, name, rating, date, review_text])

    wait = WebDriverWait(driver, 20)
    next_button = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[4]/div/div[10]/div[1]/div[2]/div/div/div/div[3]/div[2]/div/button[2]')))
    next_button.click()
    time.sleep(random.uniform(10, 20))
    print(i) #print loop


0
1
2


In [54]:
records

[['Kao Beauty & Personal Care',
  'จ***.',
  5,
  '4 สัปดาห์ก่อน',
  'ร้านค้าจัดส่งสินค้าได้รวดเร็วมาก การแพ็คสินค้าก็เรียบร้อยดีมีการใส่บับเบิ้ลแล้วใส่กล่องมาอีกชั้นนึง'],
 ['Kao Beauty & Personal Care',
  'P***i',
  5,
  '23 มิ.ย. 2024',
  'สะใจมากกันแดดขวดนี้ปริมาณ 90 ml. กดมาได้ในราคา 175 บาท ราคาเต็ม 299 บาท เบาสบายและไม่มัน, เหมาะสำหรับใช้ในชีวิตประจำวัน,'],
 ['Kao Beauty & Personal Care',
  'นิตยา เ.',
  5,
  '15 ส.ค. 2024',
  'สูตรที่ยาวนาน, อ่อนโยนต่อผิวแพ้ง่าย, รู้สึกสดชื่นบนผิว, ทิ้งความเปล่งปลั่งตามธรรมชาติและสุขภาพดี, ดีมากส่งไวใช้ประจำมีส่วนลดให้ด้วยต่ะเีงามมม'],
 ['Kao Beauty & Personal Care',
  'g***t',
  5,
  '18 ก.ค. 2024',
  'สินค้า คุณภาพดีสมราคาดีเลย การจัดส่ง ไม่ช้ามาก การแพ็คสินค้า แพ็คมาดี บริการจากบริษัทขนส่ง พอใช้ได้บริการดี โดยรวมโอเคเลยย'],
 ['Kao Beauty & Personal Care',
  'พิณพินัช บ.',
  5,
  '04 มิ.ย. 2024',
  'อ่อนโยนต่อผิวที่แพ้ง่าย, ซึมเข้าสู่ผิวได้อย่างรวดเร็ว, สูตรที่อยู่ได้นาน, ทำให้ผิวดูสุขภาพดีและเป็นธรรมชาติ,'],
 ['Kao Beauty & Personal Care',
 

In [55]:
len(records)

410

In [52]:
#เลือก filter_sort review1
wait = WebDriverWait(driver, 20)
filterStar = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[4]/div/div[10]/div[1]/div[2]/div/div/div/div[2]/div/div[2]')))
filterStar.click()

In [ ]:
#เลือก sort เป็นดาวน้อยไปดาวมาก
selectStar = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[9]/div/div/ul/li[4]')))
selectStar.click()

In [56]:
# สร้าง DataFrame และเพิ่มลำดับแถว
df = pd.DataFrame(records, columns=['shop', 'name', 'rating', 'date', 'review_text'])

# เพิ่มคอลัมน์ลำดับแถว (Row Index) โดยใช้ range
df['no'] = range(1, len(df) + 1)

# แสดง DataFrame ที่มีลำดับแถว
df = df[['no', 'shop', 'name', 'rating', 'date', 'review_text']]


In [57]:
# รับเวลาปัจจุบัน  
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")

In [58]:
# บันทึก DataFrame ลงไฟล์ CSV
csv_file = f'RawDataCSV/{output}_{current_time}.csv'
df.to_csv(csv_file, index=False)

# บันทึก DataFrame ลงไฟล์ Excel
excel_file = f'RawDataExel/{output}_{current_time}.xlsx'
df.to_excel(excel_file, index=False)